In [1]:
##imports 

import cv2
import joblib 
import pytesseract
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import confusion_matrix


In [3]:
##Needed CSV's
trainingData = pd.read_csv("train-data.csv", index_col=0) 
testingData  = pd.read_csv("test.csv", index_col=0) 

In [5]:
##batch size (started with a small no. to train the algorithm on)
batch_size = 50000

In [6]:
##vectorizing the data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(trainingData[:batch_size]['OCR text'])

In [7]:
y = trainingData[:batch_size]["Label"]

In [8]:

print(X)

(320000, 3)

In [29]:
print(vectorizer.get_feature_names())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
print(X.shape)


0


In [ ]:
##Creating pipelines for Neural Networks

from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
mnb_pipeline = Pipeline([
    ('nn', MLPClassifier())
])

In [ ]:
##Listing the testing tuning parameters

import numpy as np
from sklearn.model_selection import GridSearchCV
grid_params = {
  'nn__learning_rate_init': np.linspace(0.001, 0.005, 1),
  'nn__activation':  ["relu", "logistic", "identity"],
}
clf = GridSearchCV(mnb_pipeline, grid_params)
clf.fit(X, y)
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)


In [11]:
##Fiting the data and training it - modify tuning parameters here based on the GridsearchCV

from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(max_iter=10000, verbose=True)

clf.fit(X, y)


Iteration 1, loss = 1.85864556
Iteration 2, loss = 0.70859644
Iteration 3, loss = 0.30707895
Iteration 4, loss = 0.16819646
Iteration 5, loss = 0.11069819
Iteration 6, loss = 0.08292152
Iteration 7, loss = 0.06751830
Iteration 8, loss = 0.05855664
Iteration 9, loss = 0.05220381
Iteration 10, loss = 0.04821134
Iteration 11, loss = 0.04501211
Iteration 12, loss = 0.04267916
Iteration 13, loss = 0.04066179
Iteration 14, loss = 0.03904857
Iteration 15, loss = 0.03797243
Iteration 16, loss = 0.03672944
Iteration 17, loss = 0.03584791
Iteration 18, loss = 0.03499067
Iteration 19, loss = 0.03444515
Iteration 20, loss = 0.03360961
Iteration 21, loss = 0.03289430
Iteration 22, loss = 0.03260612
Iteration 23, loss = 0.03213587
Iteration 24, loss = 0.03184086
Iteration 25, loss = 0.03150335
Iteration 26, loss = 0.03137023
Iteration 27, loss = 0.03127812
Iteration 28, loss = 0.03060894
Iteration 29, loss = 0.03051572
Iteration 30, loss = 0.03052500
Iteration 31, loss = 0.03048173
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [25]:
##Testing data based on the fitted model

from tqdm import tqdm

#Prediction and accuracy
from sklearn.metrics import accuracy_score
documents1 = trainingData[:]["OCR text"]
y_true = trainingData[:]['Label']
input = vectorizer.transform(documents1)
prediction1= clf.predict(input)

dataframe = pd.DataFrame(list(y_true),prediction1)
index = 0
count = 0
for i in tqdm(list(y_true)):
    if i == prediction1[index]:
        count += 1
    index+=1
    
    
print("Accuracy:",accuracy_score(list(y_true),prediction1))
print(dataframe)

100%|██████████| 320000/320000 [00:00<00:00, 1163656.14it/s]


0.757165625
     0
15  15
1    1
7    7
10  10
3    3
..  ..
9    9
15  15
6    6
4    9
3    3

[320000 rows x 1 columns]
